<a href="https://colab.research.google.com/github/Emmanuel-K07/chatterbox/blob/master/stable/526_mix_comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# Nettoyage complet des dépendances conflictuelles
!pip uninstall -y torch torchvision torchaudio mediapipe pytorch-lightning numpy

# Installer Torch / TorchVision / Torchaudio (CUDA 11.8)
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 --index-url https://download.pytorch.org/whl/cu118

# Fixer numpy et mediapipe pour compatibilité
!pip install numpy==1.26.4 mediapipe==0.10.15

# Installer le reste des dépendances nécessaires (en un bloc)
!pip install xformers==0.0.20 addict yapf fvcore omegaconf --upgrade

# Cloner ComfyUI
!git clone -b v1.2 https://github.com/camenduru/ComfyUI
%cd /content/ComfyUI

# Installer requirements
!pip install -q -r requirements.txt

# Reset pour s’assurer qu’il n’y a pas de fichiers en conflit
!git reset --hard

# Cloner les preprocessors ControlNet
!git clone -b v1.2 https://github.com/camenduru/comfy_controlnet_preprocessors /content/ComfyUI/custom_nodes/comfy_controlnet_preprocessors
%cd /content/ComfyUI/custom_nodes/comfy_controlnet_preprocessors
!python install.py --no_download_ckpts
%cd /content/ComfyUI

# Configurer le tunnel Cloudflare pour accéder à l’UI
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared-linux-amd64 && chmod 777 /content/cloudflared-linux-amd64
import atexit, requests, subprocess, time, re
from random import randint
from threading import Timer
from queue import Queue

def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(
        ['/content/cloudflared-linux-amd64', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'],
        stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)",
                                   requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)

output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(8188, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(f"Accès WebUI: {tunnel_url}")

# Télécharger tous les modèles ControlNet (safetensors + yaml)
urls = [
    # safetensors
    "control_v11e_sd15_ip2p_fp16.safetensors",
    "control_v11e_sd15_shuffle_fp16.safetensors",
    "control_v11p_sd15_canny_fp16.safetensors",
    "control_v11f1p_sd15_depth_fp16.safetensors",
    "control_v11p_sd15_inpaint_fp16.safetensors",
    "control_v11p_sd15_lineart_fp16.safetensors",
    "control_v11p_sd15_mlsd_fp16.safetensors",
    "control_v11p_sd15_normalbae_fp16.safetensors",
    "control_v11p_sd15_openpose_fp16.safetensors",
    "control_v11p_sd15_scribble_fp16.safetensors",
    "control_v11p_sd15_seg_fp16.safetensors",
    "control_v11p_sd15_softedge_fp16.safetensors",
    "control_v11p_sd15s2_lineart_anime_fp16.safetensors",
    "control_v11f1e_sd15_tile_fp16.safetensors",
    # yaml
    "control_v11e_sd15_ip2p_fp16.yaml",
    "control_v11e_sd15_shuffle_fp16.yaml",
    "control_v11p_sd15_canny_fp16.yaml",
    "control_v11f1p_sd15_depth_fp16.yaml",
    "control_v11p_sd15_inpaint_fp16.yaml",
    "control_v11p_sd15_lineart_fp16.yaml",
    "control_v11p_sd15_mlsd_fp16.yaml",
    "control_v11p_sd15_normalbae_fp16.yaml",
    "control_v11p_sd15_openpose_fp16.yaml",
    "control_v11p_sd15_scribble_fp16.yaml",
    "control_v11p_sd15_seg_fp16.yaml",
    "control_v11p_sd15_softedge_fp16.yaml",
    "control_v11p_sd15s2_lineart_anime_fp16.yaml",
    "control_v11f1e_sd15_tile_fp16.yaml"
]
base = "https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/"
for file in urls:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {base}{file} -d /content/ComfyUI/models/controlnet -o {file}

# Télécharger les T2I Adapters
t2i_adapters = [
    "t2iadapter_style_sd14v1.pth",
    "t2iadapter_sketch_sd14v1.pth",
    "t2iadapter_seg_sd14v1.pth",
    "t2iadapter_openpose_sd14v1.pth",
    "t2iadapter_keypose_sd14v1.pth",
    "t2iadapter_depth_sd14v1.pth",
    "t2iadapter_color_sd14v1.pth",
    "t2iadapter_canny_sd14v1.pth",
    "t2iadapter_canny_sd15v2.pth",
    "t2iadapter_depth_sd15v2.pth",
    "t2iadapter_sketch_sd15v2.pth",
    "t2iadapter_zoedepth_sd15v1.pth"
]
for file in t2i_adapters:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {base}{file} -d /content/ComfyUI/models/controlnet -o {file}

# Télécharger RealESRGAN (Upscale)
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth -d /content/ComfyUI/models/upscale_models -o RealESRGAN_x2plus.pth

# Télécharger un checkpoint de modèle (exemple)
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/19790 -d /content/ComfyUI/models/checkpoints -o 526mixExperimental_v12.safetensors

# Lancer ComfyUI
!python main.py --dont-print-server


In [ ]:
pip install --force-reinstall torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio --index-url https://download.pytorch.org/whl/cu118
